<a href="https://colab.research.google.com/github/FabioBoccia/Progetto_ESM/blob/main/Progetto_Bozza_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%rm -r sample_data
!wget --user=corso --password=p2021corso http://www.grip.unina.it/download/corso/ffhq_real.zip
!unzip -q ffhq_real.zip
!mkdir -p ffhq/train/real
!mkdir -p ffhq/train/synthesized
!mkdir -p ffhq/test/real
!mkdir -p ffhq/test/synthesized
%cd /content/0_real/
!mv $(ls | shuf -n 2500) ../ffhq/train/real/
!mv * ../ffhq/test/real/
%cd ..
!rm -d 0_real

!pip install tensorflow_io

--2022-05-20 14:38:14--  http://www.grip.unina.it/download/corso/ffhq_real.zip
Resolving www.grip.unina.it (www.grip.unina.it)... 143.225.28.237
Connecting to www.grip.unina.it (www.grip.unina.it)|143.225.28.237|:80... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="corso"
Reusing existing connection to www.grip.unina.it:80.
HTTP request sent, awaiting response... 200 OK
Length: 4072216854 (3.8G) [application/zip]
Saving to: ‘ffhq_real.zip’

ffhq_real.zip       100%[===================>]   3.79G  17.5MB/s    in 3m 34s  

2022-05-20 14:41:51 (18.2 MB/s) - ‘ffhq_real.zip’ saved [4072216854/4072216854]

/content/0_real
/content
     |████████████████████████████████| 25.9 MB 1.4 MB/s 
     |████████████████████████████████| 2.4 MB 50.5 MB/s 
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.25.0
    Uninstalling tensorflow-io-gcs-filesystem-0.25.0:
      Successfull

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/generate/
!mkdir -p /content/tmp/
!cp * /content/tmp/
%cd /content/tmp/
!mv $(ls | shuf -n 2500) /content/ffhq/train/synthesized/
!mv * /content/ffhq/test/synthesized/

Mounted at /content/drive
/content/drive/MyDrive/generate
/content/tmp


In [2]:
!%reset -f
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import tensorflow as tf

/bin/bash: line 0: fg: no job control


In [3]:
%cd /content/
batch_size = 32
interpolation = 'bilinear'
train_dataset = tf.keras.utils.image_dataset_from_directory(
    'ffhq/train/',
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=True,
    seed=125,
    validation_split=1/5, # Validation is 1/6 of the total, so 1/5 of (total - test)
    subset='training',
    interpolation=interpolation,
    crop_to_aspect_ratio=True,
)
val_dataset = tf.keras.utils.image_dataset_from_directory(
    'ffhq/train/',
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=True,
    seed=125,
    validation_split=1/5,
    subset='validation',
    interpolation=interpolation,
    crop_to_aspect_ratio=True,
)

/content
Found 5000 files belonging to 2 classes.
Using 4000 files for training.
Found 5000 files belonging to 2 classes.
Using 1000 files for validation.


In [4]:
# Preprocessing
import tensorflow_io as tfio

d = 3
def randomGaussianBlur(x):
    k = tf.random.uniform(
        shape=[],
        minval=0.0,
        maxval=3.0,
        dtype=tf.float32
    )
    return tfio.experimental.filter.gaussian(x, d, k)

def randomRot90(x):
    k = tf.random.uniform(
        shape=[],
        minval=0,
        maxval=4,
        dtype=tf.dtypes.int32,
        seed=None,
        name=None
    )
    return tf.image.rot90(x, k)

# def preprocess(x):
#     y = tf.keras.layers.CenterCrop(224,224)(x)
#     y = tf.keras.layers.Rescaling(1./127.5, offset=-1)(x)
#     return y

def augment(x):
    y = randomGaussianBlur(x)
    y = randomRot90(x)
    return y

# val_dataset = val_dataset.map(lambda x, y: (preprocess(x), y))
train_dataset = train_dataset.map(lambda x, y: (augment(x), y))

In [5]:
# Reference: https://www.tensorflow.org/tutorials/load_data/images#configure_the_dataset_for_performance
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [11]:
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None, # hyperparameter?
)
#train_dataset = tf.keras.applications.xception.preprocess_input(train_dataset)
#val_dataset = tf.keras.applications.xception.preprocess_input(val_dataset)

In [12]:
# se presente lr scheduler, il fine tunining del lr va fatto lì
opt = tf.keras.optimizers.Nadam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
)
loss = tf.keras.losses.BinaryCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction=tf.keras.losses.Reduction.AUTO,
)
metric = tf.keras.metrics.AUC(
    num_thresholds=200,
    curve='ROC',
    summation_method='interpolation',
    name=None,
    dtype=None,
    thresholds=None,
    multi_label=False,
    num_labels=None,
    label_weights=None,
    from_logits=False # ??
)

train_after_layer = 50
for layer in base_model.layers[:train_after_layer]:
    layer.trainable = False

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(256,256,3)))
model.add(tf.keras.layers.CenterCrop(224,224))
model.add(tf.keras.layers.Rescaling(1./127.5, offset=-1))
model.add(base_model)
model.add(tf.keras.layers.GlobalMaxPooling2D())
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(
    optimizer=opt,
    loss=loss,
    metrics=[metric, 'accuracy',]
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 center_crop_2 (CenterCrop)  (None, 224, 224, 3)       0         
                                                                 
 rescaling_2 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_max_pooling2d_2 (Glo  (None, 2048)             0         
 balMaxPooling2D)                                                
                                                                 
 dense_2 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 20,863,529
Trainable params: 17,550,273
Non-trainable params: 3,313,256
__________________________________

In [13]:
# LEARNING RATE SCHEDULING - EXPONENTIAL DECAY
# lr0 - learning rate iniziale
# s - numero di step
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 ** (epoch / s)
    return exponential_decay_fn

In [14]:
%mkdir -p /content/drive/MyDrive/pesi_ffhq
epochs = 2
for i in range(51):

    # lr *0,1 dopo metà numero totale delle epoche
    exponential_decay_fn = exponential_decay(0.001, s=100)
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

    # eventuale performance scheduler
    # lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5) 

    model.fit(
        train_dataset,
        epochs=(1+i)*epochs,
        initial_epoch=i*epochs,
        verbose="auto",
        #validation_data=val_dataset,
        #validation_freq=1,
        shuffle=True,
        callbacks=[lr_scheduler] # lr scheduling
    )
    loss, AUC, accuracy = model.evaluate(val_dataset)
    model.save_weights('drive/MyDrive/pesi_ffhq/%d_%.3f_%.3f.h5' %((i+1)*epochs, loss, AUC))

Epoch 1/2
125/125 [==============================] - 106s 690ms/step - loss: 0.2736 - auc_2: 0.9551 - accuracy: 0.8835 - lr: 0.0010
Epoch 2/2
32/32 [==============================] - 21s 495ms/step - loss: 1.7993 - auc_2: 0.8408 - accuracy: 0.7520
Epoch 3/4
125/125 [==============================] - 43s 345ms/step - loss: 0.0389 - auc_2: 0.9987 - accuracy: 0.9852 - lr: 9.5499e-04
Epoch 4/4
32/32 [==============================] - 4s 140ms/step - loss: 0.5400 - auc_2: 0.9582 - accuracy: 0.9040
Epoch 5/6
125/125 [==============================] - 41s 331ms/step - loss: 0.0118 - auc_2: 0.9999 - accuracy: 0.9940 - lr: 9.1201e-04
Epoch 6/6
32/32 [==============================] - 5s 141ms/step - loss: 0.3726 - auc_2: 0.9709 - accuracy: 0.9460
Epoch 7/8
125/125 [==============================] - 41s 330ms/step - loss: 0.0058 - auc_2: 1.0000 - accuracy: 0.9977 - lr: 8.7096e-04
Epoch 8/8
32/32 [==============================] - 5s 146ms/step - loss: 0.1217 - auc_2: 0.9875 - accuracy: 0.9800
Ep

KeyboardInterrupt: ignored

In [18]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    'ffhq/test/',
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=1,
    image_size=(256, 256),
    shuffle=True,
    seed=125,
    validation_split=0,
    interpolation=interpolation,
    crop_to_aspect_ratio=True,
)

loss, AUC, accuracy = model.evaluate(test_dataset)

Found 1000 files belonging to 2 classes.
1000/1000 [==============================] - 30s 28ms/step - loss: 0.0507 - auc_2: 0.9967 - accuracy: 0.9860
